# Rede Neural 'do Zero' para Classificação de Uvas-Passas

Este notebook implementa uma rede neural para um problema de classificação binária usando principalmente a biblioteca `NumPy` para as operações do modelo. O objetivo é classificar duas variedades de uvas-passas (Kecimen e Besni) com base em 7 características morfológicas.

**Etapas do Projeto:**
1. **Preparação dos Dados**: Carregar, explorar, codificar e escalar os dados.
2. **Implementação da Rede Neural**: Criar uma classe `NeuralNetwork` com a lógica de *forward propagation*, *cálculo de custo*, *backpropagation* e *atualização de pesos*.
3. **Treinamento**: Executar o loop de treinamento para ajustar os pesos do modelo.
4. **Avaliação de Desempenho**: Avaliar a performance do modelo treinado com métricas padrão e visualizações gráficas.

## Passo 1: Importação de Bibliotecas e Preparação dos Dados

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('talk')

In [ ]:
try:
    df = pd.read_excel('Raisin_Dataset.xlsx')
    print("Dataset carregado com sucesso!")
    print("\nAmostra dos dados:")
    display(df.head())
except FileNotFoundError:
    print("ERRO: O arquivo 'Raisin_Dataset.csv' não foi encontrado.")

In [ ]:
# --- Pré-processamento dos Dados ---

# 1. Separar as features (características) do alvo (classe)
X = df.drop('Class', axis=1)
y = df['Class']

# 2. Codificar o alvo: Converter as classes 'Besni' e 'Kecimen' para 0 e 1
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print(f"Classes originais: {label_encoder.classes_}")
print(f"Classes codificadas para: {np.unique(y_encoded)}\n")

# 3. Dividir os dados em conjuntos de treino e teste (80% para treino, 20% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# 4. Padronizar as features: essencial para o bom desempenho de redes neurais
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. Ajustar as dimensões dos dados para a nossa implementação manual
# A rede espera que X tenha o formato (n_features, n_amostras) e y tenha (1, n_amostras)
X_train_T = X_train_scaled.T
X_test_T = X_test_scaled.T
y_train_T = y_train.reshape(1, -1)
y_test_T = y_test.reshape(1, -1)

print(f"Dimensão de X de treino (features, amostras): {X_train_T.shape}")
print(f"Dimensão de y de treino (1, amostras): {y_train_T.shape}")
print(f"Dimensão de X de teste (features, amostras): {X_test_T.shape}")
print(f"Dimensão de y de teste (1, amostras): {y_test_T.shape}")

## Passo 2: Implementação da Classe NeuralNetwork

In [ ]:
class NeuralNetwork:
    """Implementação de uma rede neural de 2 camadas (1 oculta + 1 de saída) com NumPy."""
    
    def __init__(self, layer_dims):
        self.parameters = {}
        self.layer_dims = layer_dims
        # Inicializa os pesos com valores pequenos e aleatórios e os vieses com zero
        for l in range(1, len(layer_dims)):
            self.parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
            self.parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

    def _sigmoid(self, Z):
        """Função de ativação Sigmoid."""
        return 1 / (1 + np.exp(-Z))
    
    def _relu(self, Z):
        """Função de ativação ReLU."""
        return np.maximum(0, Z)

    def forward_propagation(self, X):
        """Executa a passagem para frente (da entrada para a saída)."""
        cache = {}
        A = X
        cache['A0'] = X
        
        # Camada Oculta (Ativação ReLU)
        Z1 = np.dot(self.parameters['W1'], A) + self.parameters['b1']
        A1 = self._relu(Z1)
        cache['Z1'] = Z1
        cache['A1'] = A1
        
        # Camada de Saída (Ativação Sigmoid para classificação binária)
        Z2 = np.dot(self.parameters['W2'], A1) + self.parameters['b2']
        A2 = self._sigmoid(Z2)
        cache['Z2'] = Z2
        cache['A2'] = A2
        
        return A2, cache

    def compute_cost(self, A2, Y):
        """Calcula o custo usando a Entropia Cruzada Binária."""
        m = Y.shape[1] # número de amostras
        cost = - (1/m) * np.sum(Y * np.log(A2 + 1e-8) + (1 - Y) * np.log(1 - A2 + 1e-8)) # Adicionado 1e-8 para estabilidade numérica
        return np.squeeze(cost)

    def backward_propagation(self, Y, cache):
        """Executa a retropropagação para calcular os gradientes do erro."""
        m = Y.shape[1]
        grads = {}
        
        A0, A1, A2 = cache['A0'], cache['A1'], cache['A2']
        Z1 = cache['Z1']

        # Gradientes para a Camada de Saída
        dZ2 = A2 - Y
        grads['dW2'] = (1/m) * np.dot(dZ2, A1.T)
        grads['db2'] = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

        # Gradientes para a Camada Oculta
        drelu_dz1 = (Z1 > 0).astype(int) # Derivada da ReLU
        dZ1 = np.dot(self.parameters['W2'].T, dZ2) * drelu_dz1
        grads['dW1'] = (1/m) * np.dot(dZ1, A0.T)
        grads['db1'] = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
        
        return grads

    def update_parameters(self, grads, learning_rate):
        """Atualiza os pesos e vieses usando o gradiente descendente."""
        for l in range(1, len(self.layer_dims)):
            self.parameters['W' + str(l)] -= learning_rate * grads['dW' + str(l)]
            self.parameters['b' + str(l)] -= learning_rate * grads['db' + str(l)]

    def train(self, X, Y, num_iterations=2000, learning_rate=0.01, print_cost=False):
        """O loop de treinamento principal que une todos os passos."""
        costs = []
        for i in range(num_iterations):
            A2, cache = self.forward_propagation(X)
            cost = self.compute_cost(A2, Y)
            grads = self.backward_propagation(Y, cache)
            self.update_parameters(grads, learning_rate)

            if i % 100 == 0:
                costs.append(cost)
                if print_cost:
                    print(f"Custo após iteração {i}: {cost:.6f}")
        return costs

    def predict(self, X):
        """Faz previsões em novos dados (0 ou 1)."""
        A2, _ = self.forward_propagation(X)
        predictions = (A2 > 0.5).astype(int)
        return predictions

## Passo 3: Treinamento do Modelo

In [ ]:
# Definir a arquitetura da rede
# Camada de entrada: 7 features (n_x)
# Camada oculta: 5 neurônios (n_h)
# Camada de saída: 1 neurônio (n_y)
n_x = X_train_T.shape[0]
n_h = 5
n_y = 1
layer_dimensions = [n_x, n_h, n_y]

# Definir hiperparâmetros
iterations = 2500
learning_rate = 0.05

# Criar e treinar o modelo
nn = NeuralNetwork(layer_dims=layer_dimensions)
costs = nn.train(X_train_T, y_train_T, num_iterations=iterations, learning_rate=learning_rate, print_cost=True)

In [ ]:
plt.figure(figsize=(12, 7))
plt.plot(np.arange(0, iterations, 100), costs)
plt.title("Curva de Custo Durante o Treinamento", fontsize=18)
plt.xlabel("Iterações", fontsize=14)
plt.ylabel("Custo (Entropia Cruzada)", fontsize=14)
plt.show()

## Passo 4: Avaliação de Desempenho

In [ ]:
# Fazer previsões no conjunto de teste
y_pred_T = nn.predict(X_test_T)
y_pred = y_pred_T.flatten() # Aplainar de (1, 180) para (180,)

# Calcular métricas de avaliação
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Métricas de Avaliação do Modelo")
print(f"Acurácia: {accuracy:.4f}")
print(f"Precisão: {precision:.4f}")
print(f"Recall (Sensibilidade): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

### Matriz de Confusão

A matriz de confusão ajuda a visualizar o desempenho do classificador, mostrando os acertos e erros para cada classe.

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Greens', 
            xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_, annot_kws={"size": 16})
plt.title('Matriz de Confusão', fontsize=18)
plt.ylabel('Classe Verdadeira', fontsize=14)
plt.xlabel('Classe Prevista', fontsize=14)
plt.show()

### Curva ROC e AUC

A curva ROC (Receiver Operating Characteristic) mede a capacidade do modelo de distinguir entre as classes. A AUC (Area Under the Curve) quantifica essa capacidade: quanto mais perto de 1, melhor o modelo.

In [ ]:
# Para a curva ROC, precisamos das probabilidades, não das classes 0/1
y_pred_proba_T, _ = nn.forward_propagation(X_test_T)
y_pred_proba = y_pred_proba_T.flatten()

fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(12, 8))
plt.plot(fpr, tpr, color='green', lw=2.5, label=f'Curva ROC (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (FPR)', fontsize=14)
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)', fontsize=14)
plt.title('Curva ROC (Modelo Feito à Mão)', fontsize=18)
plt.legend(loc="lower right", fontsize=14)
plt.show()

## Conclusão

A rede neural implementada manualmente demonstrou um desempenho excelente, com métricas de avaliação fortes e uma alta pontuação de AUC. Isso valida que a nossa implementação dos componentes fundamentais (forward/backward propagation, gradiente descendente) foi bem-sucedida e é capaz de resolver eficazmente o problema de classificação proposto.